In [ ]:
import pandas as pd

# 1. Obtén el ID del archivo
# De tu URL: https://drive.google.com/file/d/1sNbDDK1wfmk7X0uyEKA89WHtmKoJBSjc/view
# El ID es: 1sNbDDK1wfmk7X0uyEKA89WHtmKoJBSjc
FILE_ID = '1sNbDDK1wfmk7X0uyEKA89WHtmKoJBSjc'

# 2. Crea la URL de exportación directa
DOWNLOAD_URL = f'https://drive.google.com/uc?export=download&id={FILE_ID}'

df = pd.read_csv(DOWNLOAD_URL, sep=';')

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.anova import anova_lm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
df.head()

In [ ]:
#1) Identifique observaciones que puedan considerarse problemáticas (datos atípicos,puntos de balanceo e influyentes) y analice si debe eliminarlas de su conjunto de
#datos o no, justifique. Repita la construcción del modelo de regresión si eliminó observaciones.

# Modelo inicial
X = df[["X1","X2","X3","X4"]]
X = sm.add_constant(X)
y = df["Y"]

modelo = sm.OLS(y, X).fit()

modelo.summary()

In [ ]:
influence = modelo.get_influence()
#----------------------------------------------------------------------------------
# Distancia de Cook
cooks_d = pd.Series(influence.cooks_distance[0], index=df.index)

umbral_cook = 4 / len(df)
influence_points = cooks_d[cooks_d > umbral_cook]  # ya mantiene los índices del df
indices_cook = list(influence_points.index)

print("\n" + "="*60 + "\n")
print(f"Hay {len(influence_points)} puntos influyentes.")
print("Índices influyentes en el DataFrame mediante Cook:", list(indices_cook))
print("\nValores de Cook correspondientes:\n", influence_points)


# Visualizar posibles influyentes
plt.stem(np.arange(len(cooks_d)), cooks_d)
plt.axhline(4/len(df), color="r", linestyle="--")
plt.show()
print("\n" + "="*60 + "\n")

# Gráfico de influencia
sm.graphics.influence_plot(modelo, criterion="cooks")
modelo.get_influence().resid_studentized_internal
plt.show()
print("\n" + "="*60 + "\n")
#----------------------------------------------------------------------------------
# Puntos de balanceo
influential_points = modelo.get_influence().summary_frame()
print(influential_points.head())

n = len(df)
k = 4
print(n, k)
influential_points = modelo.get_influence().summary_frame()
influential_points['leverage'] = modelo.get_influence().hat_matrix_diag
influential_points['studentized_residuals'] = modelo.get_influence().resid_studentized_internal
threshold = 2 * (k + 1) / n
outliers = influential_points[influential_points['leverage'] > threshold]
print("\n" + "="*60 + "\n")
print(f" Los puntos de balanceo son {len(outliers)}") # al parecer ninguno supera

##DDFITS
influential_points['ddfits'] = modelo.get_influence().dffits[0]
ddfits_threshold = 2 * np.sqrt((k + 1) / n)
outliers_ddfits = influential_points[abs(influential_points['ddfits']) > ddfits_threshold]
print("\n" + "="*60 + "\n")
print(f" Los puntos de balanceo con DFFITS son {len(outliers_ddfits)} equivalentes a {outliers_ddfits}") #indices 15 y 22


# índices de los puntos con DFFITS fuera del umbral:
outlier_indices_ddfits = influential_points.index[abs(influential_points['ddfits']) > ddfits_threshold]
indices_ddfits = list(outliers_ddfits.index)

#Indices de distancias de cook y dffits
indices_totales = sorted(set(indices_cook) | set(indices_ddfits))

df_clean = df.drop(index=indices_totales) # Se quitan los outliders

In [ ]:
# volvemos a construir el modelo sin outliders para comparar
X = df_clean[["X1","X2","X3","X4"]]
X = sm.add_constant(X)
y = df_clean["Y"]

modelo = sm.OLS(y, X).fit()

modelo.summary()

In [ ]:
#2) Realice la prueba de significancia del modelo, interprete.


modelo = smf.ols('Y ~ X1 + X2 + X3 + X4', data=df_clean).fit()
Anova_Table = anova_lm(modelo, typ=2)
F_value= Anova_Table['F'].iloc[0]
p_value= Anova_Table['PR(>F)'].iloc[0]
print("\n" + "="*60 + "\n")
print(Anova_Table)
print("\n" + "="*60 + "\n")

# Definimos las hipotesis
# Ho: β1 = β2 = ... = βk = 0
# Ha: Al menos un βi ≠ 0 para i = 1, 2, ..., k

if p_value < 0.05:
  print("Rechazamos la hipótesis nula, y por tanto el modelo tiene significancia.")
else:
    print("No rechazamos la hipótesis nula, y por tanto el modelo no tiene significancia.")

In [ ]:
# 3) Obtener el coeficiente de determinación y el coeficiente de determinación ajustado. Interprete.


Rsquared_Modelo_Inicial =	0.963
Rsquared_adj_Modelo_Inicial =	0.955
Rsquared_Modelo_Nuevo =	0.968
Rsquared_adj_Modelo_Nuevo =	0.961

if Rsquared_Modelo_Nuevo > Rsquared_Modelo_Inicial and Rsquared_adj_Modelo_Nuevo > Rsquared_adj_Modelo_Inicial:
  print("El nuevo modelo tiene mejor ajuste.")
else: print("El nuevo modelo no tiene mejor ajuste.")


#Ahora, para definir si continuamos con (o sin) los outliers identificados, procedemos a comparar el el R y el R-ajustado del modelo con y sin los outliers.


In [ ]:
#d) Analice si hay problemas de multicolinealidad.

corr_matrix = df.corr()
print(corr_matrix)
print("\n" + "="*60 + "\n")
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, fmt=".2f", cmap='coolwarm', square=True)
plt.title("Matriz de correlación")
plt.show()
print("\n" + "="*60 + "\n")

# Identificar variables altamente correlacionadas
threshold = 0.5
high_corr = np.where((corr_matrix.abs() > threshold) & (corr_matrix.abs() < 1))
high_corr_pairs = [(corr_matrix.index[i], corr_matrix.columns[j], corr_matrix.iloc[i, j]) for i, j in zip(*high_corr) if i < j]
print("Variables altamente correlacionadas (|corr| > 0.8):")
for var1, var2, corr_value in high_corr_pairs:
    print(f"{var1} y {var2}: {corr_value:.2f}")
print("\n" + "="*60 + "\n")

# Evaluamos multocolineidad aplicando el factor de inflación de la varianza (VIF)
vif_data = pd.DataFrame()
vif_data["Variable"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                   for i in range(X.shape[1])]
print(vif_data)
print("\n" + "="*60 + "\n")
print(f" Hay {len(vif_data[vif_data['VIF'] > 10])} variables con multicolinealidad severa equivalentes a {vif_data[vif_data['VIF'] > 10]}")

In [ ]:
#e) Realice una selección de variables por el método que prefiera, tome decisiones, explique.

from statsmodels.stats.diagnostic import het_breuschpagan, normal_ad
from scipy import stats

residuos = modelo.resid
ajustados = modelo.fittedvalues

# --- 3.1 Linealidad y homocedasticidad ---
plt.figure(figsize=(6,4))
sns.scatterplot(x=ajustados, y=residuos)
plt.axhline(0, color="red", linestyle="--")
plt.xlabel("Valores ajustados")
plt.ylabel("Residuos")
plt.title("Residuos vs valores ajustados")
plt.show()

# Test de Breusch-Pagan para heterocedasticidad
bp_test = het_breuschpagan(residuos, modelo.model.exog)
print("Breusch-Pagan p-value:", bp_test[1])  # p>0.05 → homocedasticidad

# --- 3.2 Normalidad de los residuos ---
sns.histplot(residuos, kde=True)
plt.title("Distribución de residuos")
plt.show()

sm.qqplot(residuos, line='s')
plt.title("QQ-plot de residuos")
plt.show()

# Test de Anderson-Darling
ad_stat, ad_p = normal_ad(residuos)
print("Normalidad (Anderson-Darling) p-value:", ad_p)  # p>0.05 → normalidad

# --- 3.3 Independencia de errores ---
# Test de Durbin-Watson (ya aparece en summary)
print("Durbin-Watson:", sm.stats.durbin_watson(residuos))

# --- 3.4 Observaciones influyentes ---
influence = modelo.get_influence()
cooks = influence.cooks_distance[0]

plt.figure(figsize=(6,4))
plt.stem(np.arange(len(cooks)), cooks, markerfmt=",")
plt.axhline(4/len(cooks), color="red", linestyle="--", label="Umbral 4/n")
plt.xlabel("Índice de observación")
plt.ylabel("Distancia de Cook")
plt.title("Distancias de Cook")
plt.legend()
plt.show()

In [ ]:
sns.pairplot(df)
sns.color_palette("pastel")
plt.show()

In [ ]:
import itertools
import statsmodels.api as sm

# Variables explicativas y target
X = df_clean[["X1", "X2", "X3", "X4"]]
y = df_clean["Y"]

# Modelo completo
Xc = sm.add_constant(X)
full_model = sm.OLS(y, Xc).fit()

# Función para Cp de Mallows
def mallows_cp(model, full_model, n):
    SSE_p = np.sum(model.resid ** 2)
    MSE_full = np.mean(full_model.resid ** 2)
    p = model.df_model + 1  # número de parámetros (incluye constante)
    Cp = SSE_p / MSE_full - (n - 2*p)
    return Cp

# Iterar sobre TODAS las combinaciones posibles
resultados = []

n = len(df)
variables = X.columns

for k in range(1, len(variables)+1):
    for subset in itertools.combinations(variables, k):
        Xs = sm.add_constant(X[list(subset)])
        modelo = sm.OLS(y, Xs).fit()
        Cp = mallows_cp(modelo, full_model, n)
        resultados.append({
            "Variables": subset,
            "Cp": Cp,
            "R2": modelo.rsquared,
            "AIC": modelo.aic,
            "BIC": modelo.bic
        })

# Convertir a DataFrame ordenado
df_resultados = pd.DataFrame(resultados)
df_resultados = df_resultados.sort_values(by="Cp")
df_resultados.reset_index(drop=True, inplace=True)
# parece que el modelo con X1, X2 Y X3 es el que mejor se comporta
df_resultados



In [ ]:
#Una vez elegido la combinación de variables
def ajustar_modelo(datos, y_var, x_vars):
    X = sm.add_constant(datos[x_vars])
    y = datos[y_var]
    modelo_final = sm.OLS(y, X).fit()
    return modelo_final
x_finales = ["X1", "X3", "X4"]
modelo_final = ajustar_modelo(df_clean, "Y", x_finales)

In [ ]:
# Validamos de nuevo supuestos y evaluamos el nuevo modelo
def validar_supuestos(modelo):
    residuales = modelo.resid
    fitted = modelo.fittedvalues

    print("\n--- Normalidad ---")
    shapiro_stat, shapiro_p = stats.shapiro(residuales)
    print(f"Shapiro-Wilk p={shapiro_p:.4f} → {'OK' if shapiro_p > 0.05 else 'X'}")

    print("\n--- Homocedasticidad ---")
    lm, lm_pvalue, fvalue, f_pvalue = het_breuschpagan(residuales, modelo.model.exog)
    print(f"Breusch-Pagan p={lm_pvalue:.4f} → {'OK' if lm_pvalue > 0.05 else 'X'}")

    print("\n--- Independencia ---")
    dw = sm.stats.stattools.durbin_watson(residuales)
    print(f"Durbin-Watson={dw:.2f} (cerca de 2 es ideal)")

    print("\n--- Multicolinealidad ---")
    X = modelo.model.exog
    vif = pd.DataFrame()
    vif["Variable"] = modelo.model.exog_names
    vif["VIF"] = [variance_inflation_factor(X, i) for i in range(X.shape[1])]
    print(vif)

    # Gráficos de diagnóstico
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    sns.residplot(x=fitted, y=residuales, lowess=True, ax=axes[0],
                  line_kws={'color': 'red', 'lw': 1})
    axes[0].set_title("Residuos vs Valores Ajustados")

    sm.qqplot(residuales, line='45', ax=axes[1])
    axes[1].set_title("Gráfico Q-Q")
    plt.show()
validar_supuestos(modelo_final)

In [ ]:
#Diagnostico de influencia del nuevo modelo
def diagnostico_influencia(modelo):
    infl = modelo.get_influence()
    summary = pd.DataFrame({
        "resid": modelo.resid,
        "resid_student": infl.resid_studentized_internal,
        "cooks_d": infl.cooks_distance[0],
        "hii": infl.hat_matrix_diag,
        "dffits": infl.dffits[0]
    })
    umbral_cook = 4 / len(summary)
    print(f"\nUmbral de Cook ≈ {umbral_cook:.4f}")
    print("Puntos influyentes:\n", summary[summary["cooks_d"] > umbral_cook])
    return summary
diagnostico_influencia(modelo_final)

In [ ]:
modelo_final.summary()

In [ ]:
#f) Realice una predicción utilizando el modelo seleccionado, interprete.

# Definimos nuevos datos para predicción
nuevos_datos = pd.DataFrame({
    'X1': [100, 108, 90, 110, 95, 82, 102],
    'X3': [105, 95, 110, 100, 90, 85,98],
    'X4': [100, 90, 95, 85, 80, 75, 70]
})
nuevos_datos = sm.add_constant(nuevos_datos)

predictions = modelo_final.get_prediction(nuevos_datos)
predictions_summary = predictions.summary_frame(alpha=0.05)  # Intervalo de confianza del 95%

# Mostrar las predicciones
print(f"Los valores predichos con los nuevos valores son: \n {predictions_summary}")